In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyts.image import MarkovTransitionField, GramianAngularField
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from sklearn.cluster import KMeans
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tqdm import tqdm

In [2]:
data = pd.read_csv('candles.csv').drop('Unnamed: 0', axis=1)

In [3]:
data = np.array(data)

In [4]:
def data_gen(n, data):
    
    total_prices = np.zeros((1, n))
    for i in tqdm(range(0, len(data), n*10)):
        d_close = data[i:i+n, 3]
        if i + n <= len(data):
            total_prices = np.concatenate((total_prices, d_close.reshape(1, n)), axis=0)
    total_prices = total_prices[1:]
    np.random.shuffle(total_prices)
    return total_prices

In [5]:
total_prices = data_gen(100, data)

100%|██████████| 7972/7972 [00:04<00:00, 1926.45it/s] 


In [6]:
total_prices.shape

(7972, 100)

In [7]:
gasf = GramianAngularField(image_size=100, method='difference', sample_range=(0,1), overlapping=True)
total_prices1 = gasf.fit_transform(total_prices).reshape(-1, 100, 100, 1)
gasf = GramianAngularField(image_size=100, method='summation', sample_range=(0,1), overlapping=True)
total_prices2 = gasf.fit_transform(total_prices).reshape(-1, 100, 100, 1)
X = np.concatenate([total_prices1, total_prices2], axis=3)

In [8]:
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [9]:
import tensorflow
tensorflow.compat.v1.disable_eager_execution()

In [10]:
np.random.seed(0)
nums = np.arange(len(X))
np.random.shuffle(nums)
X = X[nums]
total_prices = total_prices[nums]

In [11]:
train_data = X[:int(0.8*len(X))]
test_data = X[int(0.8*len(X)):]

In [12]:
import tensorflow

input_data = tensorflow.keras.layers.Input(shape=(100, 100, 2))

encoder = tensorflow.keras.layers.Conv2D(filters=512, kernel_size=5, activation='relu')(input_data)
encoder = tensorflow.keras.layers.MaxPooling2D(2)(encoder)
encoder = tensorflow.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu')(encoder)
encoder = tensorflow.keras.layers.MaxPooling2D(2)(encoder)
encoder = tensorflow.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu')(encoder)
encoder = tensorflow.keras.layers.MaxPooling2D(2)(encoder)
encoder = tensorflow.keras.layers.Flatten()(encoder)
encoder = tensorflow.keras.layers.Dense(64)(encoder)

In [13]:
def sample_latent_features(distribution):
    distribution_mean, distribution_variance = distribution
    batch_size = tensorflow.shape(distribution_variance)[0]
    random = tensorflow.keras.backend.random_normal(shape=(batch_size, tensorflow.shape(distribution_variance)[1]))
    return distribution_mean + tensorflow.exp(0.5 * distribution_variance) * random

In [14]:
distribution_mean = tensorflow.keras.layers.Dense(64, name='mean')(encoder)
distribution_variance = tensorflow.keras.layers.Dense(64, name='log_variance')(encoder)
latent_encoding = tensorflow.keras.layers.Lambda(sample_latent_features)([distribution_mean, distribution_variance])

In [15]:
encoder_model = tensorflow.keras.Model(input_data, latent_encoding)
encoder_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100, 100, 2  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 96, 96, 512)  26112       ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 48, 48, 512)  0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 46, 46, 256)  1179904     ['max_pooling2d[0][0]']      

In [16]:
decoder_input = tensorflow.keras.layers.Input(shape=(64))
decoder = tensorflow.keras.layers.Dense(10*10*128, activation='relu')(decoder_input)
decoder = tensorflow.keras.layers.Reshape((10, 10, 128))(decoder)
decoder = tensorflow.keras.layers.UpSampling2D(2)(decoder)
decoder = tensorflow.keras.layers.Conv2DTranspose(filters=128, kernel_size=3, activation='relu')(decoder)
decoder = tensorflow.keras.layers.UpSampling2D(2)(decoder)
decoder = tensorflow.keras.layers.Conv2DTranspose(filters=256, kernel_size=3, activation='relu')(decoder)
decoder = tensorflow.keras.layers.UpSampling2D(2)(decoder)
decoder = tensorflow.keras.layers.Conv2DTranspose(filters=512, kernel_size=3, activation='relu')(decoder)
decoder_output = tensorflow.keras.layers.Conv2DTranspose(filters=2, kernel_size=7, activation='sigmoid')(decoder)

In [17]:
decoder_model = tensorflow.keras.Model(decoder_input, decoder_output)
decoder_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64)]              0         
                                                                 
 dense_1 (Dense)             (None, 12800)             832000    
                                                                 
 reshape (Reshape)           (None, 10, 10, 128)       0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 20, 20, 128)      0         
 )                                                               
                                                                 
 conv2d_transpose (Conv2DTra  (None, 22, 22, 128)      147584    
 nspose)                                                         
                                                                 
 up_sampling2d_1 (UpSampling  (None, 44, 44, 128)      0   

In [18]:
encoded = encoder_model(input_data)
decoded = decoder_model(encoded)

In [19]:
autoencoder = tensorflow.keras.models.Model(input_data, decoded)

In [20]:
def get_loss(distribution_mean, distribution_variance):
    
    def get_reconstruction_loss(y_true, y_pred):
        reconstruction_loss = tensorflow.keras.losses.mse(y_true, y_pred)
        reconstruction_loss_batch = tensorflow.reduce_mean(reconstruction_loss)
        return reconstruction_loss_batch*100*1
    
    def get_kl_loss(distribution_mean, distribution_variance):
        kl_loss = 1 + distribution_variance - tensorflow.square(distribution_mean) - tensorflow.exp(distribution_variance)
        kl_loss_batch = tensorflow.reduce_mean(kl_loss)
        return kl_loss_batch*(-0.5)
    
    def total_loss(y_true, y_pred):
        reconstruction_loss_batch = get_reconstruction_loss(y_true, y_pred)
        kl_loss_batch = get_kl_loss(distribution_mean, distribution_variance)
        return reconstruction_loss_batch + kl_loss_batch
    
    return total_loss

In [21]:
autoencoder.compile(loss=get_loss(distribution_mean, distribution_variance), optimizer=tensorflow.keras.optimizers.Adam(lr=0.0001))
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 2)]     0         
                                                                 
 model (Functional)          (None, 64)                2328640   
                                                                 
 model_1 (Functional)        (None, 100, 100, 2)       2505090   
                                                                 
Total params: 4,833,730
Trainable params: 4,833,730
Non-trainable params: 0
_________________________________________________________________


In [22]:
autoencoder.fit(train_data, train_data, epochs=1, batch_size=128, validation_data=(test_data, test_data), callbacks=[ReduceLROnPlateau(patience=20, min_lr=0.00001), EarlyStopping(patience=40)])

Train on 6377 samples, validate on 1595 samples
6377/6377 [==============================] - 1284s 201ms/sample - loss: 33.4576 - val_loss: 25.6759 - lr: 1.0000e-04


In [23]:
autoencoder.save('autoencoder_RAW.hdf5')

In [24]:
encoder_model.save('encoder.hdf5')
decoder_model.save('decoder.hdf5')

In [25]:
class ClusteringLayer(tensorflow.keras.layers.Layer):
    '''
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.
    '''

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = tensorflow.keras.layers.InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = tensorflow.keras.layers.InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(name='clusters', shape=(self.n_clusters, input_dim), initializer='glorot_uniform') 
        
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        ''' 
        student t-distribution, as used in t-SNE algorithm.
        It measures the similarity between embedded point z_i and centroid µ_j.
                 q_ij = 1/(1+dist(x_i, µ_j)^2), then normalize it.
                 q_ij can be interpreted as the probability of assigning sample i to cluster j.
                 (i.e., a soft assignment)
       
        inputs: the variable containing data, shape=(n_samples, n_features)
        
        Return: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        '''
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1)) # Make sure all of the values of each sample sum up to 1.
        
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [26]:
clustering_layer = ClusteringLayer(2, name='clustering')(encoder_model.output)
model = Model(inputs=encoder_model.input, outputs=clustering_layer)

In [27]:
model.compile(optimizer=tensorflow.keras.optimizers.Adam(0.0001), loss='kld')

In [28]:
kmeans = KMeans(n_clusters=2, n_init=20)
y_pred = kmeans.fit_predict(encoder_model.predict(train_data))

In [29]:
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

In [30]:
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

In [31]:
loss = 0
index = 0
maxiter = 1000
update_interval = 100
index_array = np.arange(train_data.shape[0])

In [32]:
tol = 0.001

In [33]:
for ite in tqdm(range(int(maxiter))):
    if ite % update_interval == 0:
        q = model.predict(train_data, verbose=0)
        p = target_distribution(q)

    idx = index_array[index * 32: min((index+1) * 32, train_data.shape[0])]
    loss = model.train_on_batch(x=train_data[idx], y=p[idx])
    index = index + 1 if (index + 1) * 32 <= train_data.shape[0] else 0


100%|██████████| 1000/1000 [24:31<00:00,  1.47s/it] 


In [34]:
q = model.predict(test_data, verbose=0)
p = target_distribution(q)
y_pred = q.argmax(1)


In [35]:
np.save('label_MTF.npy',y_pred)

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score
s_scores = []
d_scores = []
for i in range(2, 9):
    autoencoder.load_weights('autoencoder_RAW.hdf5')
    clustering_layer = ClusteringLayer(i, name='clustering')(encoder_model.output)
    model = Model(inputs=encoder_model.input, outputs=clustering_layer)
    model.compile(optimizer=tensorflow.keras.optimizers.Adam(0.0001), loss='kld')
    kmeans = KMeans(n_clusters=i, n_init=20)
    y_pred = kmeans.fit_predict(encoder_model.predict(train_data))
    model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])
    loss = 0
    index = 0
    maxiter = 1000
    update_interval = 100
    index_array = np.arange(train_data.shape[0])
    tol = 0.001
    for ite in tqdm(range(int(maxiter))):
        if ite % update_interval == 0:
            q = model.predict(train_data, verbose=0)
            p = target_distribution(q)

        idx = index_array[index * 32: min((index+1) * 32, train_data.shape[0])]
        loss = model.train_on_batch(x=train_data[idx], y=p[idx])
        index = index + 1 if (index + 1) * 32 <= train_data.shape[0] else 0
    latent = encoder_model.predict(test_data)
    model_output = model.predict(test_data)
    labels = []
    for row in model_output:
        labels.append(np.argmax(row))
    s_scores.append(silhouette_score(latent,labels))
    d_scores.append(davies_bouldin_score(latent, labels))